### Create a df to sum up all results from all classif
    0-Good/Bad 1-Poor/Partial 2-Partial/Detailed 3-Poor/Detailed

In [7]:
from os import listdir, makedirs
from os.path import isfile, join, exists
import pandas as pd
import numpy as np
from itertools import product
from brainpipe.system import study

st = study('Olfacto')
conds, th = ['low','high'],'01'
for cond in conds:
    ###############################################################################
    path_npz = join(st.path, 'figure/0_Classif_Power_E_EpiPerf_LowHigh_'+bsl+'/')
    path2save = join(path_npz, 'Bilan_classif/')
    path_mask = join(path_npz, 'masks_visbrain'+th+'/')
    npz_form = join(path_npz, '{}_sources_{}_{}_low_high_sel_physFT_{}.npz')
    masks_vis_form = join(path_mask, '{}_mask_min{}_{}_minwin{}_th{}.npy')    
    ###############################################################################
    if not exists(path2save):
        makedirs(path2save)
    ###############################################################################

    freqs = ['2_theta','6_gamma2']#['0_VLFC','1_delta','2_theta', '3_alpha','4_beta','5_gamma1','6_gamma2']

    for freq in freqs:
        #Load all data for one classif one freq
        arch_sig = np.load(npz_form.format('All_subjects',freq, 'odor',bsl))
        print(arch_sig.files)
        #Create a df with all electrodes info
        features = ['su_codes','s_elec','s_channels','s_MAI','s_MAI_RL']
        df_elec = np.array([])
        for f in features:
            data = arch_sig[f]
            df_elec = np.vstack((df_elec,data)) if np.size(df_elec) else data
        xyz = arch_sig['s_xyz']
        x,y,z = xyz[:,0],xyz[:,1],xyz[:,2]
        df_elec = np.vstack((df_elec,x,y,z))
        df0 = pd.DataFrame(data=df_elec.T, columns=features+['x','y','z'])
        #Select max values for power and AUC scores and create Time array
        da = arch_sig['s_da']
        pow0_max, pow1_max, idx_max, da_max, time_max = np.array([]), np.array([]), np.array([]), np.array([]),np.array([])
        for elec in range(da.shape[0]):
            #print('shape da',da.shape)
            idx = [i for i,j in enumerate(da[elec]) if j == max(da[elec])][0]
            pow0, pow1 = arch_sig['s_elec_pow0'][elec][idx], arch_sig['s_elec_pow1'][elec][idx]
            time = arch_sig['s_time'][idx]
            da_elec = da[elec][idx]
            pow0_max = np.hstack((pow0_max,pow0)) if np.size(pow0_max) else pow0
            pow1_max = np.hstack((pow1_max,pow1)) if np.size(pow1_max) else pow1
            time_max = np.hstack((time_max,time)) if np.size(time_max) else time
            idx_max = np.hstack((idx_max,idx)) if np.size(idx_max) else idx
            da_max = np.hstack((da_max, da_elec)) if np.size(da_max) else da_elec
        df_data = np.concatenate((pow0_max[:,np.newaxis],pow1_max[:,np.newaxis],
                da_max[:,np.newaxis],idx_max[:,np.newaxis], time_max[:,np.newaxis]),axis=1)
        df1 = pd.DataFrame(data=df_data, columns=['p0','p1','da','idx','time'])
        df_all = pd.concat((df0,df1),axis=1)
        df_name = 'Mai_Classif_'+conds[0]+'_'+conds[1]+'_'+freq+'_all_elecs.csv'
        df_all.to_csv(path2save+df_name)

-> Olfacto loaded
['s_time', 's_channels', 's_aal', 'su_codes', 's_th_05', 's_BA', 's_labels', 's_xyz', 's_elec_pow0', 's_da', 's_elec_pow1', 's_aal_RL', 's_th_01', 's_elec', 's_MAI', 's_MAI_RL']
['s_time', 's_channels', 's_aal', 'su_codes', 's_th_05', 's_BA', 's_labels', 's_xyz', 's_elec_pow0', 's_da', 's_elec_pow1', 's_aal_RL', 's_th_01', 's_elec', 's_MAI', 's_MAI_RL']
['s_time', 's_channels', 's_aal', 'su_codes', 's_th_05', 's_BA', 's_labels', 's_xyz', 's_elec_pow0', 's_da', 's_elec_pow1', 's_aal_RL', 's_th_01', 's_elec', 's_MAI', 's_MAI_RL']
['s_time', 's_channels', 's_aal', 'su_codes', 's_th_05', 's_BA', 's_labels', 's_xyz', 's_elec_pow0', 's_da', 's_elec_pow1', 's_aal_RL', 's_th_01', 's_elec', 's_MAI', 's_MAI_RL']


### Mask significant results as function of time of the effect (wins)
    statistics p < 0.05 corrected over electrodes and time

In [18]:
from os import listdir, makedirs
from os.path import isfile, join, exists
import pandas as pd
import numpy as np
from itertools import product
conds, bsl, th,win,min_sig = ['low','high'], 'None','01',1,1
###############################################################################################
st = study('Olfacto')
path = join(st.path, 'figure/0_Classif_Power_E_EpiPerf_LowHigh_'+bsl+'/')
path_df = join(path, 'Bilan_classif/')
path2save = path_df
###############################################################################################
if not exists(path2save):
    makedirs(path2save)
##############################################################################################
freqs = ['2_theta','6_gamma2']#['0_VLFC','1_delta','2_theta', '3_alpha','4_beta','5_gamma1','6_gamma2']

for freq in freqs:
    df = pd.read_csv(path_df+'Mai_Classif_'+conds[0]+'_'+conds[1]+'_'+freq+'_all_elecs.csv')
    path_mask = join(path_npz, 'masks_stat/')
    #load data to plot and mask
    mask = np.load(masks_vis_form.format('MAI_RL',min_sig,freq,str(win),th))[:,np.newaxis]
    print(mask.shape, df.shape[1])
    mask = np.repeat(mask,df.shape[1],axis=1)
    print(mask.shape)
    if mask.dtype == bool:
        df_sel = df.mask(mask).dropna()
        df_name = 'Mai_Classif_'+conds[0]+'_'+conds[1]+'_'+freq+'_win'+str(win)+'.csv'
        df_sel.to_csv(path2save+df_name)

-> Olfacto loaded
(456, 1) 14
(456, 14)
(456, 1) 14
(456, 14)


### Mask significant results as function of time and nb of patients
    statistics p < 0.05 corrected over electrodes and time


In [34]:
from os import listdir, makedirs
from os.path import isfile, join, exists
import pandas as pd
import numpy as np
from itertools import product
conds, bsl, th,win,min_sig = ['low','high'], 'None','01',1,1
###############################################################################################
st = study('Olfacto')
path = join(st.path, 'figure/0_Classif_Power_E_EpiPerf_LowHigh_'+bsl+'/')
path_df = join(path, 'Bilan_classif/')
path2save = path_df
###############################################################################################
if not exists(path2save):
    makedirs(path2save)
##############################################################################################
freqs = ['2_theta','6_gamma2']#['0_VLFC','1_delta','2_theta', '3_alpha','4_beta','5_gamma1','6_gamma2']

df1,df2,df3,df4,df5,df6 = None, None, None,None,None,None
for freq in freqs:
    df = pd.read_csv(path_df+'Mai_Classif_'+conds[0]+'_'+conds[1]+'_'+freq+'_win'+str(win)+'.csv')
    df = df.drop(['Unnamed: 0.1'], axis=1)
    df = df.rename(index=str,columns={'Unnamed: 0':'old_idx'})
    print(df.shape, df.columns)
    
    #count and select regions with enough patients
    count = df.groupby(['s_MAI_RL']).nunique()
    count = count['su_codes']
    rois_sig1,rois_sig2,rois_sig3, rois_sig4, rois_sig5, rois_sig6 = [],[],[],[],[],[]
    for i,row in count.iteritems():
        #print('how many patients',row, 'for roi', i)
        if row >=1:
            rois_sig1.append(i)
        if row >=2:
            rois_sig2.append(i)
        if row >=3:
            rois_sig3.append(i)
        if row >=4:
            rois_sig4.append(i)
        if row >=5:
            rois_sig5.append(i)
        if row >=6:
            rois_sig6.append(i)
    print(rois_sig1,rois_sig2,rois_sig3,rois_sig4,rois_sig5,rois_sig6)
    
    # select data only in selected regions 
    df1 = df[df['s_MAI_RL'].isin(rois_sig1)] if rois_sig1 else print('No rois1')
    df2 = df[df['s_MAI_RL'].isin(rois_sig2)] if rois_sig2 else print('No rois2')
    df3 = df[df['s_MAI_RL'].isin(rois_sig3)] if rois_sig3 else print('No rois3')
    df4 = df[df['s_MAI_RL'].isin(rois_sig4)] if rois_sig4 else print('No rois4')
    df5 = df[df['s_MAI_RL'].isin(rois_sig5)] if rois_sig5 else print('No rois5')
    df6 = df[df['s_MAI_RL'].isin(rois_sig6)] if rois_sig6 else print('No rois6')
    # save data for selected regions
    df1_name = 'Mai_Classif_'+conds[0]+'_'+conds[1]+'_'+freq+'_win'+str(win)+'_patients1.csv'
    df2_name = 'Mai_Classif_'+conds[0]+'_'+conds[1]+'_'+freq+'_win'+str(win)+'_patients2.csv'
    df3_name = 'Mai_Classif_'+conds[0]+'_'+conds[1]+'_'+freq+'_win'+str(win)+'_patients3.csv'
    df4_name = 'Mai_Classif_'+conds[0]+'_'+conds[1]+'_'+freq+'_win'+str(win)+'_patients4.csv'
    df5_name = 'Mai_Classif_'+conds[0]+'_'+conds[1]+'_'+freq+'_win'+str(win)+'_patients5.csv'
    df6_name = 'Mai_Classif_'+conds[0]+'_'+conds[1]+'_'+freq+'_win'+str(win)+'_patients6.csv'
    df1.to_csv(path2save+df1_name) if df1 is not None else print('no df1 saved')
    df2.to_csv(path2save+df2_name) if df2 is not None else print('no df2 saved')
    df3.to_csv(path2save+df3_name) if df3 is not None else print('no df3 saved')
    df4.to_csv(path2save+df4_name) if df4 is not None else print('no df4 saved')
    df5.to_csv(path2save+df5_name) if df5 is not None else print('no df5 saved')
    df6.to_csv(path2save+df6_name) if df6 is not None else print('no df6 saved')

-> Olfacto loaded
(49, 14) Index(['old_idx', 'su_codes', 's_elec', 's_channels', 's_MAI', 's_MAI_RL', 'x',
       'y', 'z', 'p0', 'p1', 'da', 'idx', 'time'],
      dtype='object')
['ACC', 'Amg', 'FuG', 'HC', 'IFG', 'ITG', 'Ins', 'MFG', 'MTG', 'OFC', 'SFG', 'STG'] ['ACC', 'Amg', 'FuG', 'HC', 'IFG', 'ITG', 'Ins', 'MFG', 'MTG', 'OFC', 'STG'] ['FuG', 'IFG', 'MTG', 'OFC'] [] [] []
No rois4
No rois5
No rois6
no df4 saved
no df5 saved
no df6 saved
(156, 14) Index(['old_idx', 'su_codes', 's_elec', 's_channels', 's_MAI', 's_MAI_RL', 'x',
       'y', 'z', 'p0', 'p1', 'da', 'idx', 'time'],
      dtype='object')
['ACC', 'Amg', 'FuG', 'HC', 'IFG', 'ITG', 'Ins', 'MFG', 'MTG', 'OFC', 'PHG', 'SFG', 'STG', 'pPirT'] ['ACC', 'FuG', 'HC', 'IFG', 'MFG', 'MTG', 'OFC', 'PHG', 'SFG', 'STG'] ['FuG', 'HC', 'IFG', 'MFG', 'MTG', 'OFC', 'SFG', 'STG'] ['HC', 'IFG', 'MFG', 'MTG', 'OFC', 'STG'] ['HC', 'IFG', 'MTG', 'OFC'] []
No rois6
no df6 saved
